In [285]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [286]:
labels_training = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/labels_training_set.csv',
                              low_memory= False)
features = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/exp_features.csv', 
                              low_memory= False)
kaggle_df = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/trocafone_kaggle_test.csv', 
                              low_memory= False)

In [287]:
features['cantidad de conversiones'].fillna(0, inplace=True)
features['cantidad de veces que vio un producto'].fillna(0, inplace=True)
features['cantidad de visitas a la pagina'].fillna(0, inplace=True)
features['cantidad de checkouts'].fillna(0, inplace=True)
features['visitaron el sitio entre las 13hs y 24hs'].fillna(0, inplace=True)
features['cantidad de vistas del J5'].fillna(0, inplace=True)
features['cantidad de vistas del iphone 5s'].fillna(0, inplace=True)
features['cantidad de vistas del samsung s6 flat'].fillna(0, inplace=True)
features['cantidad de vistas del iphone 6'].fillna(0, inplace=True)
features['cantidad de vistas del iphone 6s'].fillna(0, inplace=True)
features['cantidad de vistas entre semana'].fillna(0, inplace=True)

In [288]:
test_df = pd.merge(features, labels_training, how='inner', on='person')

In [289]:
X = np.array(test_df.iloc[:, 2:13]) 
y = np.array(test_df['label']) 
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [290]:
#Instancio el xgboost
xg_reg = xgb.XGBRegressor(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.1, 
                          max_depth = 5, alpha = 10, n_estimators = 10)

In [291]:
#Entreno
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [292]:
#prediccion
preds = xg_reg.predict(X_test)

In [293]:
#Error de las predicciones
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.262009


In [294]:
pred_df = pd.merge(features, kaggle_df, how='inner', on='person')

In [295]:
X_train = np.array(test_df.iloc[:, 2:13]) 
y_train = np.array(test_df['label']) 
X_test = np.array(pred_df.iloc[:, 2:13]) 

In [296]:
xg_reg.fit(X_train,y_train)
pred_df['label'] = abs(xg_reg.predict(X_test))

In [297]:
#Error de las predicciones finales
final_preds_error = np.sqrt(mean_squared_error(y_test, preds))
print("Error: %f" % (final_preds_error))

Error: 0.262009


In [298]:
pred_df.head(20)

,Unnamed: 0,person,cantidad de conversiones,cantidad de veces que vio un producto,cantidad de visitas a la pagina,cantidad de checkouts,visitaron el sitio entre las 13hs y 24hs,cantidad de vistas del J5,cantidad de vistas del iphone 5s,cantidad de vistas del samsung s6 flat,cantidad de vistas del iphone 6,cantidad de vistas del iphone 6s,cantidad de vistas entre semana,label
0,0,4886f805,0.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.195620
1,2,0297fc1e,0.0,404.0,95.0,7.0,64.0,0.0,1.0,0.0,213.0,213.0,81.0,0.274552
2,3,2d681dd8,0.0,13.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.199223
3,4,cccea85e,0.0,739.0,22.0,1.0,18.0,0.0,2.0,1.0,0.0,0.0,20.0,0.233743
4,5,4c8a8b93,0.0,177.0,20.0,2.0,13.0,0.0,21.0,0.0,83.0,83.0,10.0,0.227525
5,7,29ebb414,0.0,8.0,4.0,1.0,4.0,0.0,0.0,0.0,1.0,1.0,2.0,0.205797
6,11,3dc1950f,0.0,595.0,30.0,3.0,24.0,0.0,10.0,16.0,21.0,21.0,30.0,0.235788
7,12,8ea4c165,0.0,97.0,7.0,3.0,6.0,3.0,0.0,9.0,0.0,0.0,4.0,0.224341
8,14,d8cfe234,1.0,39.0,4.0,3.0,4.0,0.0,0.0,8.0,0.0,0.0,4.0,0.221580
9,17,d6bc64df,0.0,155.0,7.0,2.0,2.0,0.0,18.0,0.0,8.0,8.0,4.0,0.215428


In [299]:
#Armo el submit
out_df = pred_df[['person', 'label']]
out_df.set_index('person', inplace=True)
out_df.head()

,label
person,
4886f805,0.195620
0297fc1e,0.274552
2d681dd8,0.199223
cccea85e,0.233743
4c8a8b93,0.227525


In [300]:
out_df.to_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/submit.csv')